In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dgl
from dgl.nn import ChebConv
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import numpy.linalg as alg
import os
from tqdm import tqdm
from functools import reduce
import argparse


def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)

res = 10


# clustering
# generally, we fix the outcome of clustering
clusters = nx_comm.louvain_communities(g, seed = 10, resolution=res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}


for i in g.nodes:
    g.nodes[i]["deg"] = g.degree[i]

rev_cluster_sizes = cluster_sizes[::-1]
rev_clusters = clusters[::-1]
rev_cluster_sizes = np.array(rev_cluster_sizes)


In [3]:
dgl_G = dgl.from_networkx(g)

In [4]:
ramps = [0.02, 0.05, 0.1, 0.25, 0.5]
repeat_num = 1000

In [5]:

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = dgl.nn.ChebConv(2, 16, 2)
        self.conv2 = dgl.nn.ChebConv(16, 16, 1)
        self.conv3 = dgl.nn.ChebConv(16, 1, 1)
        
    def forward(self, g, features):
        x = self.conv1(g, features)
        x = self.conv2(g, x)
        x = self.conv3(g, x)
        return x
        

In [6]:
def po_linear_model_sqrt_expo(graph, alpha=1, beta=1, sigma=0.1, gamma=1):    
    for i in graph.nodes:
        graph.nodes[i]["y"] = alpha + beta * graph.nodes[i]["z"] + sigma * np.random.normal() +  gamma * np.sqrt(sum([graph.nodes[ngbr]['z'] for ngbr in graph[i]])/graph.degree[i])

In [ ]:
# unit-level cr
GATE_hat_list = np.zeros((repeat_num, len(ramps)))

for seed in tqdm(range(repeat_num)):    
    np.random.seed(seed)     
    rollout_index = np.random.uniform(0, 1, size=(num_nodes,))   
    
    for num_step in range(len(ramps)):
        print(f"Seed: {seed}")
        torch.manual_seed(2)
        p_list = [0, ramps[num_step]]                
        g_feat_list = []
        y_list = []
        
        for p in p_list:
            z = (rollout_index < np.quantile(rollout_index, p))            
            nx.set_node_attributes(g, 0, "z")            
            nx.set_node_attributes(g, {unit:1 for unit in range(num_nodes) if z[unit]}, "z")            
            po_linear_model_sqrt_expo(g, gamma = 1)                        
            g_feat_list.append(torch.tensor([[g.nodes[n]['z'], g.nodes[n]['deg']] for n in g.nodes], dtype=torch.float))
            y_list.append(torch.tensor([[g.nodes[n]['y']] for n in g.nodes], dtype=torch.float).reshape(-1))
    
        # Instantiate the model and optimizer
        model = GCN()    
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
        
        # Train the model
        for epoch in range(400):
            for i in range(len(p_list)):
                model.train()
                optimizer.zero_grad()
                out = model(dgl_G, g_feat_list[i])
                loss = F.mse_loss(out.squeeze(), y_list[i])  
                loss.backward()
                optimizer.step()
            
                if epoch % 50 == 0:
                    print(f'Epoch {epoch}, Treat Prop: {p_list[i]:.2f}, Loss: {loss:.4f}')
    
        print(loss.item())
        g_feat = g_feat_list[0]
    
        g_feat[:,0] = 0
        global_control_pred = model(dgl_G, g_feat).detach().numpy()
    
        g_feat[:,0] = 1
        global_treat_pred = model(dgl_G, g_feat).detach().numpy()
    
        GATE_hat = global_treat_pred.mean() - global_control_pred.mean()
    
        GATE_hat_list[seed, num_step] = GATE_hat
        
    torch.save(GATE_hat_list, "Result/SQRT_incre_2step_CR_unit.pkl")


  0%|          | 0/1000 [00:00<?, ?it/s]

Seed: 0


/root/miniconda3/envs/myconda/lib/python3.10/site-packages/dgl/nn/pytorch/conv/chebconv.py:108: DGLWarning: lambda_max is not provided, using default value of 2.  Please use dgl.laplacian_lambda_max to compute the eigenvalues.
  dgl_warning(


Epoch 0, Treat Prop: 0.00, Loss: 8.1175
Epoch 0, Treat Prop: 0.02, Loss: 2.3009
Epoch 50, Treat Prop: 0.00, Loss: 0.1527
Epoch 50, Treat Prop: 0.02, Loss: 0.2013
Epoch 100, Treat Prop: 0.00, Loss: 0.0332
Epoch 100, Treat Prop: 0.02, Loss: 0.0590
Epoch 150, Treat Prop: 0.00, Loss: 0.0210
Epoch 150, Treat Prop: 0.02, Loss: 0.0367
Epoch 200, Treat Prop: 0.00, Loss: 0.0195
Epoch 200, Treat Prop: 0.02, Loss: 0.0315
Epoch 250, Treat Prop: 0.00, Loss: 0.0185
Epoch 250, Treat Prop: 0.02, Loss: 0.0288
Epoch 300, Treat Prop: 0.00, Loss: 0.0178
Epoch 300, Treat Prop: 0.02, Loss: 0.0273
Epoch 350, Treat Prop: 0.00, Loss: 0.0173
Epoch 350, Treat Prop: 0.02, Loss: 0.0263
0.02578709088265896
Seed: 0
Epoch 0, Treat Prop: 0.00, Loss: 8.1203
Epoch 0, Treat Prop: 0.05, Loss: 2.3151
Epoch 50, Treat Prop: 0.00, Loss: 0.1744
Epoch 50, Treat Prop: 0.05, Loss: 0.2893
Epoch 100, Treat Prop: 0.00, Loss: 0.0480
Epoch 100, Treat Prop: 0.05, Loss: 0.1130
Epoch 150, Treat Prop: 0.00, Loss: 0.0321
Epoch 150, Treat P

  0%|          | 1/1000 [00:24<6:42:05, 24.15s/it]

...

0.06987994909286499
Seed: 217
Epoch 0, Treat Prop: 0.00, Loss: 8.1243
Epoch 0, Treat Prop: 0.02, Loss: 2.2927
Epoch 50, Treat Prop: 0.00, Loss: 0.1547
Epoch 50, Treat Prop: 0.02, Loss: 0.2011
Epoch 100, Treat Prop: 0.00, Loss: 0.0331
Epoch 100, Treat Prop: 0.02, Loss: 0.0595
Epoch 150, Treat Prop: 0.00, Loss: 0.0214
Epoch 150, Treat Prop: 0.02, Loss: 0.0371
Epoch 200, Treat Prop: 0.00, Loss: 0.0198
Epoch 200, Treat Prop: 0.02, Loss: 0.0316
Epoch 250, Treat Prop: 0.00, Loss: 0.0188
Epoch 250, Treat Prop: 0.02, Loss: 0.0288
Epoch 300, Treat Prop: 0.00, Loss: 0.0180
Epoch 300, Treat Prop: 0.02, Loss: 0.0272
Epoch 350, Treat Prop: 0.00, Loss: 0.0175
Epoch 350, Treat Prop: 0.02, Loss: 0.0262
0.02561248652637005
Seed: 217
Epoch 0, Treat Prop: 0.00, Loss: 8.1079
Epoch 0, Treat Prop: 0.05, Loss: 2.3115
Epoch 50, Treat Prop: 0.00, Loss: 0.1720
Epoch 50, Treat Prop: 0.05, Loss: 0.2873
Epoch 100, Treat Prop: 0.00, Loss: 0.0473
Epoch 100, Treat Prop: 0.05, Loss: 0.1117
Epoch 150, Treat Prop: 0.00,

 22%|██▏       | 218/1000 [1:24:51<5:04:15, 23.34s/it]

0.0709376409649849
Seed: 218
Epoch 0, Treat Prop: 0.00, Loss: 8.1078
Epoch 0, Treat Prop: 0.02, Loss: 2.2938
Epoch 50, Treat Prop: 0.00, Loss: 0.1502
Epoch 50, Treat Prop: 0.02, Loss: 0.2022
Epoch 100, Treat Prop: 0.00, Loss: 0.0336
Epoch 100, Treat Prop: 0.02, Loss: 0.0610
Epoch 150, Treat Prop: 0.00, Loss: 0.0220
Epoch 150, Treat Prop: 0.02, Loss: 0.0380
Epoch 200, Treat Prop: 0.00, Loss: 0.0205
Epoch 200, Treat Prop: 0.02, Loss: 0.0326
Epoch 250, Treat Prop: 0.00, Loss: 0.0196
Epoch 250, Treat Prop: 0.02, Loss: 0.0299
Epoch 300, Treat Prop: 0.00, Loss: 0.0188
Epoch 300, Treat Prop: 0.02, Loss: 0.0284
Epoch 350, Treat Prop: 0.00, Loss: 0.0182
Epoch 350, Treat Prop: 0.02, Loss: 0.0274
0.02682841569185257
Seed: 218
Epoch 0, Treat Prop: 0.00, Loss: 8.1219
Epoch 0, Treat Prop: 0.05, Loss: 2.3129
Epoch 50, Treat Prop: 0.00, Loss: 0.1748
Epoch 50, Treat Prop: 0.05, Loss: 0.2851
Epoch 100, Treat Prop: 0.00, Loss: 0.0485
Epoch 100, Treat Prop: 0.05, Loss: 0.1127
Epoch 150, Treat Prop: 0.00, 

 22%|██▏       | 219/1000 [1:25:14<5:03:12, 23.29s/it]

0.06960244476795197
Seed: 219
Epoch 0, Treat Prop: 0.00, Loss: 8.1206
Epoch 0, Treat Prop: 0.02, Loss: 2.3074
Epoch 50, Treat Prop: 0.00, Loss: 0.1517
Epoch 50, Treat Prop: 0.02, Loss: 0.2025
Epoch 100, Treat Prop: 0.00, Loss: 0.0333
Epoch 100, Treat Prop: 0.02, Loss: 0.0611
Epoch 150, Treat Prop: 0.00, Loss: 0.0212
Epoch 150, Treat Prop: 0.02, Loss: 0.0370
Epoch 200, Treat Prop: 0.00, Loss: 0.0198
Epoch 200, Treat Prop: 0.02, Loss: 0.0317
Epoch 250, Treat Prop: 0.00, Loss: 0.0189
Epoch 250, Treat Prop: 0.02, Loss: 0.0291
Epoch 300, Treat Prop: 0.00, Loss: 0.0182
Epoch 300, Treat Prop: 0.02, Loss: 0.0275
Epoch 350, Treat Prop: 0.00, Loss: 0.0176
Epoch 350, Treat Prop: 0.02, Loss: 0.0266
0.02578703500330448
Seed: 219
Epoch 0, Treat Prop: 0.00, Loss: 8.1160
Epoch 0, Treat Prop: 0.05, Loss: 2.3209
Epoch 50, Treat Prop: 0.00, Loss: 0.1735
Epoch 50, Treat Prop: 0.05, Loss: 0.2827
Epoch 100, Treat Prop: 0.00, Loss: 0.0469
Epoch 100, Treat Prop: 0.05, Loss: 0.1106
Epoch 150, Treat Prop: 0.00,